In [179]:
# Librerias necesarias
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [180]:
# Cargar los datos
df = pd.read_excel("../data/0 TimeLog (original, no modificar).xlsx")

In [181]:
# Copio para no modificar el original
tl = df.copy()

In [182]:
# Agrego LOS a cada fila y reordeno
tl["LOS"] = (tl["TF"] - tl["TI"])
tl = tl[["ID", "MS_GRD", "UBICACIÓN", "TI", "TF", "LOS", "HOSPITAL", "UNIDAD"]]

# Lista para guardar filas nuevas
new_rows = []

# Recorrer filas consecutivas
for i in range(len(tl) - 1):
    row_current = tl.iloc[i]
    row_next = tl.iloc[i + 1]

    # Verificar condiciones
    same_id = row_current['ID'] == row_next['ID']
    same_ms = row_current['MS_GRD'] == row_next['MS_GRD']
    time_gap = row_current['TF'] < row_next['TI']
    same_hospital = row_current['HOSPITAL'] == row_next['HOSPITAL']

    if same_id and same_ms and time_gap:
        # Crear fila intermedia
        new_row = {
            'ID': row_current['ID'],
            'MS_GRD': row_current['MS_GRD'],
            'UBICACIÓN': f"{row_current['UBICACIÓN']} -> {row_next['UBICACIÓN']}",
            'TI': row_current['TF'],
            'TF': row_next['TI'],
            'LOS': row_next['TI'] - row_current['TF'],
            'HOSPITAL': row_current['HOSPITAL'],
            'UNIDAD': row_current['UNIDAD']
        }
        new_rows.append((i + 1, new_row))  # guardar con índice de inserción

# Lista para guardar todas las filas nuevas (dicts)
new_rows_list = [fila[1] for fila in new_rows]

# Convertir todo a un DataFrame una sola vez
new_rows_df = pd.DataFrame(new_rows_list)

# Concatenar en una sola pasada
tl = pd.concat([tl, new_rows_df], ignore_index=True)

# Reordenar filas para meter filas entremedio
tl = tl.sort_values(["ID", "TI"])

# Resetear index pra quie se vea ordenado
tl = tl.reset_index(drop=True)

In [183]:
# Ubicaciones
ubicaciones = list(tl["UBICACIÓN"].unique())
ubicaciones.sort()
ubicaciones

['Hospital_1_ED',
 'Hospital_1_GA',
 'Hospital_1_ICU',
 'Hospital_1_ICU -> Hospital_1_OR',
 'Hospital_1_ICU -> Hospital_1_SDU_WARD',
 'Hospital_1_OR',
 'Hospital_1_OR -> Hospital_1_ICU',
 'Hospital_1_OR -> Hospital_1_SDU_WARD',
 'Hospital_1_SDU_WARD',
 'Hospital_1_SDU_WARD -> Hospital_1_ICU',
 'Hospital_1_SDU_WARD -> Hospital_1_OR',
 'Hospital_2_ED',
 'Hospital_2_GA',
 'Hospital_2_ICU',
 'Hospital_2_ICU -> Hospital_2_OR',
 'Hospital_2_ICU -> Hospital_2_SDU_WARD',
 'Hospital_2_OR',
 'Hospital_2_OR -> Hospital_2_ICU',
 'Hospital_2_OR -> Hospital_2_SDU_WARD',
 'Hospital_2_SDU_WARD',
 'Hospital_2_SDU_WARD -> Hospital_2_ICU',
 'Hospital_2_SDU_WARD -> Hospital_2_OR',
 'Hospital_3_ED',
 'Hospital_3_GA',
 'Hospital_3_ICU',
 'Hospital_3_ICU -> Hospital_3_OR',
 'Hospital_3_ICU -> Hospital_3_SDU_WARD',
 'Hospital_3_OR',
 'Hospital_3_OR -> Hospital_3_ICU',
 'Hospital_3_OR -> Hospital_3_SDU_WARD',
 'Hospital_3_SDU_WARD',
 'Hospital_3_SDU_WARD -> Hospital_3_ICU',
 'Hospital_3_SDU_WARD -> Hospital_3_

In [184]:
tl.head(5)

,ID,MS_GRD,UBICACIÓN,TI,TF,LOS,HOSPITAL,UNIDAD
0,1,5,WL_WL,12,384,372,WL,WL
1,1,5,Hospital_1_GA,384,468,84,Hospital_1,GA
2,1,5,Hospital_1_ICU,468,552,84,Hospital_1,ICU
3,1,5,Hospital_1_ICU -> Hospital_1_SDU_WARD,552,576,24,Hospital_1,ICU
4,1,5,Hospital_1_SDU_WARD,576,768,192,Hospital_1,SDU_WARD


In [185]:
# Función con que unidades a considerar

def los_case_mix_report(tl: pd.DataFrame, unidades: list = ["ICU", "OR", "SDU_WARD"]):
    # Proporción de GRDs entre todos los casos (P_j)
    variable = tl.groupby('ID')['MS_GRD'].max().reset_index()["MS_GRD"].value_counts(normalize=True).sort_index().reset_index()
    P_j = {}
    for index, row in variable.iterrows():
        P_j[int(row['MS_GRD'])] = row['proportion']

    # LOS estando ya hospitalizado promedio por GRD (A_j)
    v1 = tl[tl["UNIDAD"].isin(unidades)]
    v2 = v1.groupby('ID').agg({
        'LOS': 'sum',
        'MS_GRD': 'min'
    }).reset_index().groupby("MS_GRD")["LOS"].mean().reset_index()
    A_j = {}
    for index, row in v2.iterrows():
        A_j[int(row['MS_GRD'])] = row['LOS']

    # LOS promedio por hospital_i y GRD_j (a_i_j)
    a_i_j = {}
    for numero in range(1,4):
        v1 = tl[tl["HOSPITAL"] == f"Hospital_{numero}"]
        v2 = v1[v1["UNIDAD"].isin(unidades)]
        v3 = v2.groupby('ID').agg({
            'LOS': 'sum',
            'MS_GRD': 'min'
        }).reset_index().groupby("MS_GRD")["LOS"].mean().reset_index()
        for index, row in v3.iterrows():
            a_i_j[(numero,int(row['MS_GRD']))] = row['LOS']

    # Proporción de GRDs en cada hospital (p_i_j)
    p_i_j = {}

    for numero in range(1,4):
        # Filtro por hospital
        v1 = tl[tl["HOSPITAL"] == f"Hospital_{numero}"]
        # Filtro que este en alguna de esas unidades, cuando ya estan aceptados por el hospital
        v2 = v1[v1["UNIDAD"].isin(unidades)]
        v3 = v2.groupby('ID')['MS_GRD'].max().reset_index()["MS_GRD"].value_counts(normalize=True).sort_index().reset_index()
        for index, row in v3.iterrows():
                p_i_j[(numero,int(row['MS_GRD']))] = row['proportion']

    # LOS promedio total
    v1 = tl[tl["UNIDAD"].isin(unidades)]
    v2 = v1.groupby('ID').agg({'LOS': 'sum'}).reset_index()
    mean_los = v2["LOS"].mean()

    # Average LOS by hospital
    mean_los_i = {}
    for numero in range(1,4):
        v1 = tl[tl["HOSPITAL"] == f"Hospital_{numero}"]
        v2 = v1[v1["UNIDAD"].isin(unidades)]
        v3 = v2.groupby('ID').agg({
            'LOS': 'sum',
            'MS_GRD': 'min'
        }).reset_index()
        mean_los_i[numero] = v3["LOS"].mean()

    # Case mix adjusted LOS
    cmal_i = {}
    for i in range(1,4):
        suma = 0
        for j in range(1, 9):
            suma += a_i_j[(i, j)] * P_j[j]
        cmal_i[i] = suma

    # LOS weighted case mix
    lwcm_i = {}
    for i in range(1,4):
        suma = 0
        for j in range(1, 9):
            suma += A_j[j] * p_i_j[(i, j)]
        lwcm_i[i] = suma

    # Convertir todo a un DataFrame una sola vez
    new_rows = []
    for i in range(1,4):
        new_row = {
            'HOSPITAL': i,
            'AVERAGE LOS': mean_los_i[i],
            'CASE MIX ADJUSTED LOS': cmal_i[i],
            'LOS WEIGHTED CASE MIX': lwcm_i[i]
        }
        new_rows.append(new_row)
    new_rows_df = pd.DataFrame(new_rows)

    new_rows_df["AVERAGE LOS INDEX"] = new_rows_df["AVERAGE LOS"] / mean_los
    new_rows_df["LOS INDEX"] = new_rows_df["CASE MIX ADJUSTED LOS"] / mean_los
    new_rows_df["CASE MIX INDEX"] = new_rows_df["LOS WEIGHTED CASE MIX"] / mean_los

    new_rows_df["AVERAGE LOS DIFFER"] = new_rows_df["AVERAGE LOS"] - mean_los
    new_rows_df["DIFFER DUE TO LOS"] = new_rows_df["CASE MIX ADJUSTED LOS"] - mean_los
    new_rows_df["DIFFER DUE TO CASE MIX"] = new_rows_df["LOS WEIGHTED CASE MIX"] - mean_los

    new_rows_df["INTERACT DIFFER"] = new_rows_df["AVERAGE LOS DIFFER"] - new_rows_df["DIFFER DUE TO LOS"] - new_rows_df["DIFFER DUE TO CASE MIX"]

    return new_rows_df

# Ejemplo de uso
#unidades = ["ICU", "OR", "SDU_WARD", "GA"]
unidades = ["ICU", "OR", "SDU_WARD"]
report_df = los_case_mix_report(tl, unidades)
display(report_df)


,HOSPITAL,AVERAGE LOS,CASE MIX ADJUSTED LOS,LOS WEIGHTED CASE MIX,AVERAGE LOS INDEX,LOS INDEX,CASE MIX INDEX,AVERAGE LOS DIFFER,DIFFER DUE TO LOS,DIFFER DUE TO CASE MIX,INTERACT DIFFER
0,1,309.654573,293.038532,307.723614,1.016514,0.961968,1.010175,5.030433,-11.585609,3.099473,13.516569
1,2,296.776073,290.421790,296.849523,0.974237,0.953377,0.974478,-7.848067,-14.202351,-7.774617,14.128901
2,3,307.748436,289.821874,309.586704,1.010256,0.951408,1.016291,3.124295,-14.802266,4.962564,12.963998


In [227]:
# Matriz de transición
for hospital in range(1,4):    
    for grd in range(1, 9):
        v3 = tl[(tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])) & 
            (tl["MS_GRD"] == grd) & 
            (tl["HOSPITAL"] == f"Hospital_{hospital}")].sort_values(["ID", "TI"])

        transiciones = {}

        for i in range(len(v3) - 1):
            row_current = v3.iloc[i]
            row_next = v3.iloc[i + 1]

            # Verificar condiciones
            same_id = row_current['ID'] == row_next['ID']
            actual = row_current['UNIDAD']
            siguiente = row_next['UNIDAD']
            distinta = actual != siguiente
            if same_id and distinta:
                if (actual, siguiente) in transiciones:
                    transiciones[(actual, siguiente)] += 1

                else:
                    transiciones[(actual, siguiente)] = 1

            # Solo para cuando esta en SDU_WARD al ser nodo de termino
            elif not same_id and actual == "SDU_WARD":
                if (actual, actual) in transiciones:
                    transiciones[(actual, actual)] += 1
                else:
                    transiciones[(actual, actual)] = 1



        unidades = ["OR","ICU", "SDU_WARD"]
        num = len(unidades) + 1
        matriz = []
        for i in range(num):
            matriz.append([0] * num)

        for i in range(1, num):
            matriz[0][i] = unidades[i - 1]
            matriz[i][0] = unidades[i - 1]

        for i in range(1, num):
            for j in range(1, num):
                if (matriz[i][0], matriz[0][j]) in transiciones:
                    matriz[i][j] = transiciones[(matriz[i][0], matriz[0][j])]
                else:
                    matriz[i][j] = 0

        for i in range(1, num):
            total = sum(matriz[i][1:])
            for j in range(1, num):
                    matriz[i][j] = round(matriz[i][j]/total, 3)
        
        matriz[0][0] = ""
        matriz[1][1] = 0
        matriz[2][2] = 0

        print(f"Hospital {hospital} for MS_GRD: {grd}")
        # display(pd.DataFrame(matriz))
        display(matriz)

Hospital 1 for MS_GRD: 1


[['', 'OR', 'ICU', 'SDU_WARD'],
 ['OR', 0, 0.932, 0.068],
 ['ICU', 0.084, 0, 0.916],
 ['SDU_WARD', 0.014, 0.069, 0.917]]

Hospital 1 for MS_GRD: 2


[['', 'OR', 'ICU', 'SDU_WARD'],
 ['OR', 0, 0.927, 0.073],
 ['ICU', 0.095, 0, 0.905],
 ['SDU_WARD', 0.015, 0.04, 0.945]]

Hospital 1 for MS_GRD: 3


...

[['', 'OR', 'ICU', 'SDU_WARD'],
 ['OR', 0, 0.903, 0.097],
 ['ICU', 0.09, 0, 0.91],
 ['SDU_WARD', 0.009, 0.042, 0.949]]

Hospital 3 for MS_GRD: 7


[['', 'OR', 'ICU', 'SDU_WARD'],
 ['OR', 0, 0.974, 0.026],
 ['ICU', 0.121, 0, 0.879],
 ['SDU_WARD', 0.021, 0.059, 0.92]]

Hospital 3 for MS_GRD: 8


[['', 'OR', 'ICU', 'SDU_WARD'],
 ['OR', 0, 0.892, 0.108],
 ['ICU', 0.024, 0, 0.976],
 ['SDU_WARD', 0.012, 0.028, 0.96]]